# predict image acquisition parameters (IAPs) directly from breast MR images

In [1]:
import os
if os.getcwd() != 'workspace' and os.path.exists('workspace'):
    os.chdir('workspace')

In [2]:
from src.dataset import *
from src.utils import *
from src.vizutils import *
from src.IAP_model import get_total_criterion

import os
import random
from tqdm import tqdm
import datetime

# torch
import torch
from torch import nn
from torchvision import transforms
from torchvision.models import resnet18

# GPUs
device_ids = [1] # indices of devices for models, data and otherwise
os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(str(i) for i in device_ids)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
print('running on {}'.format(device))

# set random seed
seed = 1337
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

running on cuda


In [3]:
# data and model choice
dataset_name = 'dbc_by_scanner'
labeling = 'all'

train_size = 10000
val_size = 2000
test_size = 2000

model = resnet18

checkpoint_paths = {
}

# training options
train = True
batch_size_factors = {
          'resnet18' : 512
}
checkpoint_path_prev = None
train_with_augmentations = False
save_checkpoints = True
checkpoint_setting = 'incremental'
#checkpoint_setting = 'best'


In [4]:
# option for other experiments
test_this_IAP_only = None

In [5]:
# load dataset and loader (note: this will take a minute or so to run)
img_size = 224
train_batchsize = batch_size_factors[model.__name__] * len(device_ids)
eval_batchsize = 64

if train_with_augmentations:
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(size=img_size)
    ])
    print('training with augmentations')
else:
    train_transform = transforms.Compose([])

trainset, valset, testset = get_datasets(dataset_name, 
                                train_size=train_size, 
                                val_size=val_size,
                                test_size=test_size,
                                labeling=labeling,
                                return_filenames=True,
                                different_cases_for_train_val_test=True,
                                regress_dense_IAPs=True
                            )


trainloader = DataLoader(trainset, 
                        batch_size=train_batchsize, 
                        shuffle=True)
valloader = DataLoader(valset, 
                    batch_size=eval_batchsize)
testloader = DataLoader(testset, 
                    batch_size=eval_batchsize)

using non-default labeling: all
there are missing features for 5 patients


/workspace/github_public/src/dataset.py:41: RuntimeWarning: invalid value encountered in divide
  img = img.astype(float) * 255. / img.max()
/workspace/github_public/src/dataset.py:43: RuntimeWarning: invalid value encountered in cast
  img = img.astype(np.uint8)


In [ ]:
# load model
net = model()
# fix first lyr
make_netinput_onechannel(net, model)

try:
    all_feature_names = trainset.dataset.dataset.all_feature_names
except AttributeError:
    all_feature_names = trainset.dataset.all_feature_names
num_output_features = np.array(list(all_feature_names.values())).sum()
print(num_output_features)
change_num_output_features(net, model, num_output_features)

net = net.to(device)
net = torch.nn.DataParallel(net, device_ids = range(len(device_ids)))

47


In [ ]:
# set up custom loss
total_criterion = get_total_criterion(all_feature_names, device)

In [ ]:
if train:
    # training
    epochs = 100
    checkpoint_dir = "saved_models/feature_pred_all/{}".format(dataset_name)

    optimizer = torch.optim.Adam(net.parameters(),
                                lr= 0.001,# default=0.001
                                weight_decay=0.0001     
                            )

    start_epoch = 0
    best_val_loss = np.inf
    for epoch in range(start_epoch, epochs):
        net.train()
        print("Epoch {}:".format(epoch))

        total_examples = 0

        train_loss = 0
        train_feature_losses = {feature_name : 0 for feature_name in all_feature_names.keys()}

        # train for one epoch
        for batch_idx, (inputs, targets, _) in tqdm(enumerate(trainloader), total=len(trainloader.dataset)//train_batchsize):
            inputs = inputs.to(device)

            # apply transformations
            inputs = train_transform(inputs)

            # reset gradients
            optimizer.zero_grad()

            # inference
            outputs = net(inputs)

            # backprop
            total_loss, feature_losses, _ = total_criterion(outputs, targets)
            total_loss.backward()

            # iterate
            optimizer.step()

            train_loss += total_loss
            train_feature_losses = {feature_name : train_feature_losses[feature_name] + feature_losses[feature_name] for feature_name, _ in feature_losses.items()}

        # results
        avg_loss = train_loss / (batch_idx + 1)
        print("Training loss: %.4f" %(avg_loss))
        # print("training loss per feature: ", {feature_name : train_feature_losses[feature_name] / (batch_idx + 1) for feature_name, feature_loss in train_feature_losses.items()})

        print(datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
        print("Validation...")
        total_examples = 0

        net.eval()

        val_loss = 0
        val_feature_losses = {feature_name : 0 for feature_name in all_feature_names.keys()}

        # for accuracy computation
        total_examples = 0
        total_feature_correct_classification_examples = None 
        with torch.no_grad():
            for batch_idx, (inputs, targets, _) in enumerate(valloader):
                # Copy inputs to device
                inputs = inputs.to(device)
                # Generate output from the DNN.
                outputs = net(inputs)
                
                total_loss, feature_losses, feature_correct_classification_examples = total_criterion(outputs, targets)

                # print(feature_correct_classification_examples)

                val_loss += total_loss
                val_feature_losses = {feature_name : val_feature_losses[feature_name] + feature_losses[feature_name] for feature_name, feature_loss in feature_losses.items()}

                # for accuracy computation at the end
                total_examples += eval_batchsize 

                if not total_feature_correct_classification_examples:
                    # initialize correct example counts only for classification features
                    total_feature_correct_classification_examples = {feature_name : 0 for feature_name in feature_correct_classification_examples.keys()}
                # add counts to totals
                total_feature_correct_classification_examples = {feature_name : total_feature_correct_classification_examples[feature_name] + feature_correct_classification_examples[feature_name] for feature_name, _ in feature_correct_classification_examples.items()}

                # print('running total correct example counts:', total_feature_correct_classification_examples, '\n')

        avg_loss = val_loss / len(valloader)
        print("validation loss: %.4f" %(avg_loss))

        # show metrics per feature (convert CE losses to accΩuracies)
        # avg all losses over batch
        val_metrics_per_feature = {feature_name : val_feature_losses[feature_name] / (batch_idx + 1) for feature_name, feature_loss in val_feature_losses.items()}

        # convert CE losses to accuracies
        for feature_name in feature_correct_classification_examples.keys():
            avg_acc = total_feature_correct_classification_examples[feature_name] / total_examples
            val_metrics_per_feature[feature_name] = avg_acc

        print("validation metrics per feature: ", val_metrics_per_feature, '\n')

        # Save for checkpoint
        if save_checkpoints:
            if (avg_loss < best_val_loss and checkpoint_setting == 'best') or ((epoch % 5 == 0) and checkpoint_setting == 'incremental'):
                best_val_loss = avg_loss
                if not os.path.exists(checkpoint_dir):
                    os.makedirs(checkpoint_dir)
                print("Saving ...")
                state = {'net': net.state_dict(),
                            'epoch': epoch}

                # delete older checkpoint
                if checkpoint_setting == 'best' and checkpoint_path_prev:
                    os.remove(checkpoint_path_prev)

                # save new checkpoint
                best_val_metric = avg_loss
                checkpoint_path = "{}_{}_{}_{}_{}_{}_best.h5".format(model.__name__,
                    len(trainset), len(valset), best_val_metric, epoch, labeling)
                checkpoint_path = os.path.join(checkpoint_dir, checkpoint_path)
                torch.save(state, checkpoint_path)

                checkpoint_path_prev = checkpoint_path


Epoch 0:


40it [01:16,  1.91s/it]                                                                        


Training loss: 112117.1875
02/12/2023, 19:29:00
Validation...
validation loss: 100422.0781
validation metrics per feature:  {'Manufacturer': 0.6617943548387096, 'Manufacturer Model Name': 0.2908266129032258, 'Scan Options': 0.3855846774193548, 'Patient Position During MRI': 0.8175403225806451, 'Field Strength (Tesla)': 0.4586693548387097, 'Contrast Agent': 0.5942540322580645, 'Acquisition Matrix': 0.28931451612903225, 'Slice Thickness ': 0.21339766440852995, 'Flip Angle \n': 6.710176298695226, 'FOV Computed (Field of View) in cm ': 100406.08896169355, 'TE (Echo Time)': 0.3030691146850586, 'TR (Repetition Time)': 0.7886245500656867} 

Saving ...
Epoch 1:


40it [01:11,  1.80s/it]                                                                        


Training loss: 90209.4766
02/12/2023, 19:30:26
Validation...
validation loss: 72842.9219
validation metrics per feature:  {'Manufacturer': 0.626008064516129, 'Manufacturer Model Name': 0.32711693548387094, 'Scan Options': 0.3125, 'Patient Position During MRI': 0.8155241935483871, 'Field Strength (Tesla)': 0.5352822580645161, 'Contrast Agent': 0.5942540322580645, 'Acquisition Matrix': 0.2782258064516129, 'Slice Thickness ': 0.19173745618712518, 'Flip Angle \n': 0.9044667020920785, 'FOV Computed (Field of View) in cm ': 72832.64868951614, 'TE (Echo Time)': 0.3087952665744289, 'TR (Repetition Time)': 0.7191014068741952} 

Epoch 2:


...

Training loss: 8.0447
02/12/2023, 20:06:14
Validation...
validation loss: 166.7526
validation metrics per feature:  {'Manufacturer': 0.9788306451612904, 'Manufacturer Model Name': 0.8805443548387096, 'Scan Options': 0.9667338709677419, 'Patient Position During MRI': 0.9369959677419355, 'Field Strength (Tesla)': 0.9097782258064516, 'Contrast Agent': 0.7273185483870968, 'Acquisition Matrix': 0.7852822580645161, 'Slice Thickness ': 0.15675002769116433, 'Flip Angle \n': 0.07333740964531898, 'FOV Computed (Field of View) in cm ': 164.2292236820344, 'TE (Echo Time)': 0.01704455085939938, 'TR (Repetition Time)': 0.09491270756529223} 

Epoch 98:


40it [00:20,  1.98it/s]                                                                        


Training loss: 8.6437
02/12/2023, 20:06:40
Validation...
validation loss: 166.2743
validation metrics per feature:  {'Manufacturer': 0.9778225806451613, 'Manufacturer Model Name': 0.78125, 'Scan Options': 0.9470766129032258, 'Patient Position During MRI': 0.9077620967741935, 'Field Strength (Tesla)': 0.9037298387096774, 'Contrast Agent': 0.7071572580645161, 'Acquisition Matrix': 0.6920362903225806, 'Slice Thickness ': 0.17015775653623766, 'Flip Angle \n': 0.07377566445258356, 'FOV Computed (Field of View) in cm ': 163.2487315516318, 'TE (Echo Time)': 0.014097631818825198, 'TR (Repetition Time)': 0.09421218547128862} 

Epoch 99:


40it [00:19,  2.02it/s]                                                                        


Training loss: 8.9224
02/12/2023, 20:07:05
Validation...
validation loss: 183.4317
validation metrics per feature:  {'Manufacturer': 0.9778225806451613, 'Manufacturer Model Name': 0.8659274193548387, 'Scan Options': 0.9601814516129032, 'Patient Position During MRI': 0.9112903225806451, 'Field Strength (Tesla)': 0.8825604838709677, 'Contrast Agent': 0.7384072580645161, 'Acquisition Matrix': 0.748991935483871, 'Slice Thickness ': 0.15096055812412693, 'Flip Angle \n': 0.07773031486618903, 'FOV Computed (Field of View) in cm ': 180.66366774036038, 'TE (Echo Time)': 0.011915003940943749, 'TR (Repetition Time)': 0.11579222664717705} 



# Testing

In [ ]:
checkpoint_path = 'saved_models/feature_pred_all/dbc_by_scanner/...' # fill in here
net.load_state_dict(torch.load(checkpoint_path)['net'])

print("Testing...")
total_examples = 0

net.eval()

test_loss = 0
test_feature_losses = {feature_name : 0 for feature_name in all_feature_names.keys()}

# regression results
TE_gt = []
TE_pred = []
TR_gt = []
TR_pred = []

# for accuracy computation
total_examples = 0
total_feature_correct_classification_examples = None 
topk_total_feature_correct_classification_examples = None 
with torch.no_grad():
    for batch_idx, (inputs, targets, _) in enumerate(testloader):
        # Copy inputs to device
        inputs = inputs.to(device)
        # Generate output from the DNN.
        outputs = net(inputs)

        # print(outputs)

        # store regression results individually
        TE_pred += outputs[:, -2: -1].flatten().tolist()
        TR_pred += outputs[:, -1:].flatten().tolist()
        TE_gt += targets['TE (Echo Time)'].float().tolist()
        TR_gt += targets['TR (Repetition Time)'].float().tolist()
        
        total_loss, feature_losses, feature_correct_classification_examples, topk_feature_correct_classification_counts = total_criterion(outputs, targets, get_topk_counts=True)

        # print(feature_correct_classification_examples)

        test_loss += total_loss
        test_feature_losses = {feature_name : test_feature_losses[feature_name] + feature_losses[feature_name] for feature_name, feature_loss in feature_losses.items()}

        # for accuracy computation at the end
        total_examples += eval_batchsize 

        if not total_feature_correct_classification_examples:
            # initialize correct example counts only for classification features
            total_feature_correct_classification_examples = {feature_name : 0 for feature_name in feature_correct_classification_examples.keys()}
        # add counts to totals
        total_feature_correct_classification_examples = {feature_name : total_feature_correct_classification_examples[feature_name] + feature_correct_classification_examples[feature_name] for feature_name, _ in feature_correct_classification_examples.items()}

        # print('running total correct example counts:', total_feature_correct_classification_examples, '\n')

        # top k stuff
        if not topk_total_feature_correct_classification_examples:
            # initialize correct example counts only for classification features
            topk_total_feature_correct_classification_examples = {}
            for k in topk_feature_correct_classification_counts.keys():
                topk_total_feature_correct_classification_examples[k] = {feature_name : 0 for feature_name in topk_feature_correct_classification_counts[k].keys()}
        
        # add counts to totals
        for k in topk_feature_correct_classification_counts.keys():
            topk_total_feature_correct_classification_examples[k] = {feature_name : topk_total_feature_correct_classification_examples[k][feature_name] + topk_feature_correct_classification_counts[k][feature_name] for feature_name, _ in topk_feature_correct_classification_counts[k].items()} 

avg_loss = test_loss / len(testloader)
print("test loss: %.4f" %(avg_loss))

# show metrics per feature (convert CE losses to accuracies)
# avg all losses over batch
test_metrics_per_feature = {feature_name : test_feature_losses[feature_name] / (batch_idx + 1) for feature_name, feature_loss in test_feature_losses.items()}

# convert CE losses to accuracies
for feature_name in feature_correct_classification_examples.keys():
    avg_acc = total_feature_correct_classification_examples[feature_name] / total_examples
    test_metrics_per_feature[feature_name] = avg_acc

print("test metrics per feature: ", test_metrics_per_feature, '\n')

print("top k accuracies:")
for k, topk_correct_classification_counts in topk_feature_correct_classification_counts.items():
    print("k = {}:".format(k))
    print(topk_correct_classification_counts)
    for feature_name in topk_correct_classification_counts.keys():
        avg_acc = topk_total_feature_correct_classification_examples[k][feature_name] / total_examples
        print(feature_name, avg_acc)


RuntimeError: Error(s) in loading state_dict for DataParallel:
	size mismatch for module.fc.weight: copying a param with shape torch.Size([96, 512]) from checkpoint, the shape in current model is torch.Size([47, 512]).
	size mismatch for module.fc.bias: copying a param with shape torch.Size([96]) from checkpoint, the shape in current model is torch.Size([47]).

# Visualizations

## TE and TR

In [ ]:
viz_dir = 'visualizations'
# fontsize_lrg = 26
# fontsize_med = 20
# fontsize_sm = 16
fontsize_lrg = 16
fontsize_med = 12
fontsize_sm = 10
# visualize regression errors
# prediction vs. g.t.
colors = ['b', 'r']
fig, axs = plt.subplots(1, 2, figsize=(8,4))
for idx, feature_name in enumerate(['TE (Echo Time)', 'TR (Repetition Time)']):
    gt = TE_gt if feature_name == 'TE (Echo Time)' else TR_gt
    pred = TE_pred if feature_name == 'TE (Echo Time)' else TR_pred

    xline = np.linspace(np.min(gt)-0.25, np.max(gt)+.25, 1000)
    axs[idx].plot(xline, xline, 'k-')
    axs[idx].scatter(gt, pred, s=10, alpha=0.35, c=colors[idx])
    axs[idx].set_xlabel('true value (ms)', fontsize=fontsize_med)
    if idx == 0:
        axs[idx].set_ylabel('predicted value (ms)', fontsize=fontsize_med)
    axs[idx].set_title('{}'.format(feature_name))#, round(test_metrics_per_feature[feature_name], 4)))
    axs[idx].set_ylim(np.min(xline), np.max(xline))
    axs[idx].grid()

# fig.suptitle("Prediction vs. Ground truth: TE and TR")
fig.tight_layout()
plt.savefig(os.path.join(viz_dir, 'regression_results.pdf'.format(feature_name)))
plt.show()

## Example predictions (visualization)

In [ ]:
def get_prediction_names(patient_preds):
    clinical_features_cats_path = 'data/dbc/maps/clinical_feature_categories.csv'
    # convert a predicted category index for an IAP into its name
    patient_preds_named = {}
    clinical_features = pd.read_csv(clinical_features_cats_path)
    for feature_name, feature_pred in patient_preds.items():
        # if continuous, just add unit
        if feature_name in ['TE (Echo Time)', 'TR (Repetition Time)']:
            patient_preds_named[feature_name] = '{} ms'.format(round(float(feature_pred), 3))
            continue
        else:
            feature_pred = int(feature_pred)
            feature_map = clinical_features[feature_name].iloc[0]
            feature_map = feature_map.split(',')
            feature_map = [x.strip() for x in feature_map]
            feature_map = {int(x.split('=')[1]): x.split('=')[0] for x in feature_map}
            named_pred = feature_map[feature_pred] 

            # shorten name
            if feature_name == 'Manufacturer':
                named_pred = named_pred.split(' ')[0]

            if feature_name == 'Scan Options':
               named_pred = str(feature_pred)

            # add unit if necessary
            if feature_name == 'Field Strength (Tesla)':
                named_pred = '{} T'.format(named_pred)
            elif feature_name == 'Slice Thickness ':
                named_pred = '{} mm'.format(named_pred)
            elif feature_name == 'Flip Angle \n':
                named_pred = '{} deg'.format(named_pred)
            elif feature_name == 'FOV Computed (Field of View) in cm ':
                named_pred = '{} cm'.format(named_pred)

            
            patient_preds_named[feature_name] = named_pred

    return patient_preds_named

num_examples = 6
fig, axs = plt.subplots(2, num_examples+1, figsize=(2 * num_examples, 4.5), height_ratios=[1, 1])

# feature name labels
all_feature_names_short = []

# shorten if needed
for feature_name in all_feature_names.keys():
    name = feature_name
    if feature_name == 'Manufacturer Model Name':
        name = 'Model'
    elif feature_name == 'Patient Position During MRI':
        name = 'Patient Position'
    elif feature_name == 'Slice Thickness ':
        name = 'Slice Thickness'
    elif feature_name == 'Flip Angle \n':
        name = 'Flip Angle'
    elif feature_name == 'FOV Computed (Field of View) in cm ':
        name = 'FOV Computed'
    elif feature_name == 'Field Strength (Tesla)':
        name = 'Field Strength'
    elif feature_name == 'TR (Repetition Time)':
        name = 'TR (Rep. Time)'

    
    
    all_feature_names_short.append(name)


txt = ['{}:'.format(n) for n in all_feature_names_short]
txt = '\n' + '\n'.join(txt)
axs[0, 0].text(0, 0.1, "Predicted IAPs", fontweight='semibold', fontsize=11)
axs[0, 0].set_axis_off()
axs[1, 0].set_axis_off()
axs[1, 0].text(0, 0, txt, fontstyle='italic')


# get predictions
net.eval()

testloader.shuffle = True

patient_IDs = []
with torch.no_grad():
    for batch_idx, (inputs, targets, filenames) in enumerate(testloader):
        # if len(patient_IDs) < num_examples:
        #     break

        # Copy inputs to device
        inputs = inputs.to(device)
        # Generate output from the DNN.
        outputs = net(inputs)
        total_loss, feature_losses, feature_correct_classification_examples, predictions = total_criterion(outputs, targets, return_predictions=True)

        #for idx in range(num_examples):
        for idx in range(inputs.shape[0]):
            col_idx = len(patient_IDs) + 1
            patient_ID = filenames[idx].split('-')[2].replace('.png', '')
            patient_preds = {feature_name : predictions[feature_name][idx].cpu().numpy() for feature_name in predictions.keys()}
            
            # ensure different patients shown
            if patient_ID in patient_IDs:
                continue

            patient_IDs.append(patient_ID)
            # show scan
            try:
                axs[0, col_idx].set_axis_off()
                #axs[0, idx].set_title("Patient {}".format(patient_ID))
                axs[0, col_idx].imshow(inputs[idx,0,:,:].cpu().numpy(), cmap='gray')

                # show predictions
                # get named predictions
                #patient_preds_named = testset.dataset.dataset.get_prediction_names(patient_preds)
                patient_preds_named = get_prediction_names(patient_preds)

                # are they correct?
                for feature_name in patient_preds_named.keys():
                    #print(patient_preds[feature_name], targets[feature_name][idx])
                    correct = False
                    if feature_name in ['TE (Echo Time)', 'TR (Repetition Time)']:
                        # "Correct" if relative error <%1 for continuous IAPs
                        pred = float(patient_preds[feature_name])
                        gt = float(targets[feature_name][idx])
                        if ((pred - gt) / gt) < 0.02:
                            correct = True
                    
                    else:
                        if patient_preds[feature_name] == targets[feature_name][idx].item():
                            correct = True

                    if correct:
                        patient_preds_named[feature_name] = '{} {}'.format(patient_preds_named[feature_name], u'\u2713')
                    else:
                        patient_preds_named[feature_name] = '{} {}'.format(patient_preds_named[feature_name], u'\u2717')

                preds_txt = '\n'.join([str(v) for v in patient_preds_named.values()])

                # show
                axs[1, col_idx].set_axis_off()
                axs[1, col_idx].text(0, 0, preds_txt)

                print(filenames[idx])
            
            except IndexError:
                break


plt.savefig(os.path.join(viz_dir, 'predictions.pdf'), bbox_inches='tight')
plt.show()
